In [1]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv
!pip3 install langchain-chroma


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
# from pydantic import BaseModel, ImportString, PydanticUserError, Field
from langchain_chroma import Chroma

# other modules and packages
import os
import tempfile
import streamlit as st
import pandas as pd
import numpy
from dotenv import load_dotenv

/Users/sgnclexus/Development/rag_llms/myenv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## Define our LLM

In [17]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm.invoke("Tell me one example of how to achieve to read an understand a book?")

AIMessage(content='One effective way to read and understand a book is to use the **active reading technique**. Here’s how to do it:\n\n1. **Preview the Book**: Before diving in, skim the table of contents, introduction, and conclusion (if available). Look at any chapter headings, subheadings, and summary sections to get a sense of the book’s structure and main ideas.\n\n2. **Take Notes**: As you read, jot down key points, questions, and summaries in your own words. Use margins to write notes or highlight important passages. This process helps you engage with the material actively, reinforcing your understanding.\n\n3. **Ask Questions**: While reading, pause to ask yourself questions about the material. What is the author trying to convey? How does this information connect to what I already know? This critical thinking approach can deepen comprehension.\n\n4. **Discuss with Others**: If possible, join a book club or find a friend to discuss the book with. Sharing insights and perspectiv

## Process PDF Document 

### Load PDF Document

In [5]:
loader = PyPDFLoader("data/LeyPropiedadCondomino.pdf")
pages = loader.load()
pages 

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)
Ignoring wrong pointing object 588 0 (offset 0)


[Document(metadata={'source': 'data/LeyPropiedadCondomino.pdf', 'page': 0, 'page_label': '1'}, page_content='Distribución gratuita\nLey de \nPropiedad \nen Condominio\nde Inmuebles\npara el Distrito\nFederal'),
 Document(metadata={'source': 'data/LeyPropiedadCondomino.pdf', 'page': 1, 'page_label': '2'}, page_content='Desde el 1 de enero de 1999, la Ley de Propiedad \nen Condominio de Inmuebles para el Distrito \nFederal estableció las normas generales \npara la constitución, funcionamiento y extinción \nde este tipo de régimen de propiedad. \nEste marco regulatorio instituyó a la Procuraduría \nSocial del Distrito Federal (PROSOC) \ncomo autoridad en la materia y brazo ejecutor \nde dicha ley.\nEl 11 de febrero de 2000 entraron en vigor varias \nreformas y adiciones a esta ley que fortalecieron \ny ampliaron las facultades de la PROSOC, pero \ntambién a las administraciones condominales, \ndotándolas de mayores atribuciones.\nLa presente edición de la Ley de Propiedad \nen Condominio 

### Split document

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=500,
                                               length_function=len,
                                               separators=["\n\n","\n"," "])
chunks = text_splitter.split_documents(pages)
chunks

[Document(metadata={'source': 'data/LeyPropiedadCondomino.pdf', 'page': 0, 'page_label': '1'}, page_content='Distribución gratuita\nLey de \nPropiedad \nen Condominio\nde Inmuebles\npara el Distrito\nFederal'),
 Document(metadata={'source': 'data/LeyPropiedadCondomino.pdf', 'page': 1, 'page_label': '2'}, page_content='Desde el 1 de enero de 1999, la Ley de Propiedad \nen Condominio de Inmuebles para el Distrito \nFederal estableció las normas generales \npara la constitución, funcionamiento y extinción \nde este tipo de régimen de propiedad. \nEste marco regulatorio instituyó a la Procuraduría \nSocial del Distrito Federal (PROSOC) \ncomo autoridad en la materia y brazo ejecutor \nde dicha ley.\nEl 11 de febrero de 2000 entraron en vigor varias \nreformas y adiciones a esta ley que fortalecieron \ny ampliaron las facultades de la PROSOC, pero \ntambién a las administraciones condominales, \ndotándolas de mayores atribuciones.\nLa presente edición de la Ley de Propiedad \nen Condominio 

### Create embeddings

In [7]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_type=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()
test_vector = embedding_function.embed_query("dog")

In [8]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

{'score': 0.1745443723078154}

In [ ]:
evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop")

### Create a vector database

In [9]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)


    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=chunks,
                                        ids=list(unique_ids),
                                        embedding=embedding_function,
                                        persist_directory="vectorstore")

    # vectorstore.persist()
    # vectorstore._persist_directory()

    return vectorstore

In [10]:
 # create vectorstore
vectorstore = create_vectorstore(chunks=chunks,
                                 embedding_function=embedding_function,
                                 vectorstore_path="vectorstore_chroma"
                                 )

 ## 2. Query for relevant data

In [11]:
# load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

In [12]:
# create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("¿Cual es la duracion maxima en la que puede permanecer el administrador interno o externo en su cargo?")
relevant_chunks

[Document(id='ff300a19-3b8c-5db1-98cb-7aa28943ed64', metadata={'page': 72, 'source': 'data/LeyPropiedadCondomino.pdf'}, page_content='LEY CONDOMINAL\n71\ntegrantes de los comités señalados en la fracción IX \ndel artículo 16 de esta ley, en un plazo no mayor de \n30 días naturales.\n(REFORMADO PRIMER PÁRRAFO, G.O.D.F. 13 DE ENERO \nDE 2015)\nArtículo 41.-  Cuando la Asamblea General elija y \ndecida contratar servicios profesionales para su Ad-\nministración, determinará las bases para garantizar \nel buen desempeño del cargo, nombrando al Comi-\nté de Vigilancia para celebrar el contrato correspon-\ndiente, conforme a la legislación aplicable.\nEl Administrador profesional, tendrá un plazo no \nmayor a quince días naturales, a partir de la cele -\nbración de la Asamblea General para entregar al Co-\nmité de Vigilancia la fianza correspondiente.\n(REFORMADO, G.O.D.F. 13 DE ENERO DE 2015)\nArtículo 42.-  El Administrador durará en su cargo \nun año, siempre que a consideración del Comit

In [13]:
PROMPT_TEMPLATE = """
    Tu eres miembro del comite de vigilacion por lo que es necesario conocer todas las normas 
    de la ley de propiedad en condominios, responde que no sabes sino tines las respuesta

    {context}

    ---

    Responde las preguntas de acuerdo al contexto proporcionado: {question}
"""



In [14]:
# concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question="¿Cual es el plazo maximo que puede permanecer el administrador en un condominio?")
print(prompt)

Human: 
    Tu eres miembro del comite de vigilacion por lo que es necesario conocer todas las normas 
    de la ley de propiedad en condominios, responde que no sabes sino tines las respuesta

    LEY CONDOMINAL
71
tegrantes de los comités señalados en la fracción IX 
del artículo 16 de esta ley, en un plazo no mayor de 
30 días naturales.
(REFORMADO PRIMER PÁRRAFO, G.O.D.F. 13 DE ENERO 
DE 2015)
Artículo 41.-  Cuando la Asamblea General elija y 
decida contratar servicios profesionales para su Ad-
ministración, determinará las bases para garantizar 
el buen desempeño del cargo, nombrando al Comi-
té de Vigilancia para celebrar el contrato correspon-
diente, conforme a la legislación aplicable.
El Administrador profesional, tendrá un plazo no 
mayor a quince días naturales, a partir de la cele -
bración de la Asamblea General para entregar al Co-
mité de Vigilancia la fianza correspondiente.
(REFORMADO, G.O.D.F. 13 DE ENERO DE 2015)
Artículo 42.-  El Administrador durará en su cargo 


## 3. Generate responses

In [18]:
llm.invoke(prompt)

AIMessage(content='El plazo máximo que puede permanecer el administrador en un condominio es de tres años, ya que puede ser reelecto en dos periodos consecutivos más, siempre y cuando se cumpla con lo establecido en el contrato y sea aprobado por el Comité de Vigilancia.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 1208, 'total_tokens': 1263, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-dcfd0e50-c9ed-41d0-b4d7-90a477780c97-0', usage_metadata={'input_tokens': 1208, 'output_tokens': 55, 'total_tokens': 1263, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Using Langchain Expression Language

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm 
            )

rag_chain.invoke("¿Cual es el plazo maximo que puede permanecer el administrador en un condominio?")

AIMessage(content='Según el contexto proporcionado, el plazo máximo que puede permanecer el Administrador en un condominio es de un año. Sin embargo, dos de sus integrantes pueden ser reelectos por un periodo consecutivo adicional, y posteriormente, pueden participar en otros periodos no consecutivos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 1078, 'total_tokens': 1135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-7172c19f-8d29-47b1-af39-500fc44430c4-0', usage_metadata={'input_tokens': 1078, 'output_tokens': 57, 'total_tokens': 1135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 're

## Generate structured responses

In [20]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Respuesta a la pregunta")
    sources: str = Field(description="Fragmentos donde se demuestra la respuesta")
    reasoning: str = Field(description="Explica basado en las fuentes")

class ExtractedInfo(BaseModel):
    """Extracted information about the document"""
    # max_time: AnswerWithSources
    max_time: AnswerWithSources
    # paper_title: AnswerWithSources
    # paper_summary: AnswerWithSources

# schema = ExtractedInfo.model_json_schema()

In [21]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo)
)

rag_chain.invoke("¿Cual es el plazo maximo que puede permanecer el administrador en un condominio?")

/Users/sgnclexus/Development/rag_llms/myenv/lib/python3.13/site-packages/langchain_openai/chat_models/base.py:1362: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


ExtractedInfo(max_time=AnswerWithSources(answer='El plazo máximo que puede permanecer el administrador en un condominio es de un año, aunque dos de los integrantes del Comité de Administración pueden ser reelectos por un periodo consecutivo más.', sources='LEY CONDOMINAL, Artículo 20', reasoning='La Ley Condominial establece que la duración del Comité de Administración del Conjunto Condominal será de un año, y se menciona que dos de sus integrantes pueden ser reelectos por un periodo consecutivo más.'))

## Transform response into a dataframe

In [22]:
structured_response = rag_chain.invoke("¿Cual es el plazo maximo que puede permanecer el administrador en un condominio?")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and source
answer_row = []
source_row = []
reasoning_row = []


for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer','sources','reasoning'])
structured_response_df


,max_time
answer,El plazo máximo que puede permanecer el admini...
sources,LEY CONDOMINAL - Artículo 20
reasoning,"De acuerdo a la Ley Condominial, el Comité de ..."
